# Prerequisites
- Install python
- Create a virtual environment
- use pip to install the requirements

# Strands Introduction

Strands SDK is an open source python framework developed by AWS for building AI Agents. It is designed to simplify agent development by taking a model first approach. This allows developers to focus on defining goals and tools.

Strands enables the developer to call AWS Bedrock with minimal code. For tooling, Strands provides a common library `strands_tools` as well the ability to easily create tools. 

In [1]:
from strands import Agent, tool
from strands_tools import calculator, current_time
@tool
def letter_counter(word: str, letter: str) -> int:
    """
    Count occurrences of a specific letter in a word.

    Args:
        word (str): The input word to search in
        letter (str): The specific letter to count

    Returns:
        int: The number of occurrences of the letter in the word
    """
    if not isinstance(word, str) or not isinstance(letter, str):
        return 0

    if len(letter) != 1:
        raise ValueError("The 'letter' parameter must be a single character")

    return word.lower().count(letter.lower())
agent = Agent(tools=[calculator, current_time, letter_counter])

# Ask the agent a question that uses the available tools
message = """
I have 3 requests for you to do:

1. What is the time right now?
2. Calculate 3111696 / 74088
3. Tell me how many letter R's are in the word "strawberry" 🍓
"""
agent(message)

I'll help you with all three requests! Let me get the answers for you.
Tool #1: current_time

Tool #2: calculator

Tool #3: letter_counter


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 3111696 / 74088     │                                                                            │
│  │ Result    │ 42                  │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Here are the answers to your three requests:

1. **Current time**: It's currently **2025-09-26T18:55:20** UTC (6:55:20 PM on September 26th, 2025)

2. **3111696 ÷ 74088**: The answer is exactly **42** 

3. **Letter R's in "strawberry"** 🍓: There are **3** letter R's in the word "strawberry"

Interesting that your division problem resulted in the answer to life, the universe, and everything! 😄

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Here are the answers to your three requests:\n\n1. **Current time**: It\'s currently **2025-09-26T18:55:20** UTC (6:55:20 PM on September 26th, 2025)\n\n2. **3111696 ÷ 74088**: The answer is exactly **42** \n\n3. **Letter R\'s in "strawberry"** 🍓: There are **3** letter R\'s in the word "strawberry"\n\nInteresting that your division problem resulted in the answer to life, the universe, and everything! 😄'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'current_time': ToolMetrics(tool={'toolUseId': 'tooluse_nXlx-DLDQrqqMrsveWtXaQ', 'name': 'current_time', 'input': {}}, call_count=1, success_count=1, error_count=0, total_time=0.0011570453643798828), 'letter_counter': ToolMetrics(tool={'toolUseId': 'tooluse_G2SgxZbYTvO4mCY0Z58YGQ', 'name': 'letter_counter', 'input': {'word': 'strawberry', 'letter': 'r'}}, call_count=1, success_count=1, error_count=0, total_time=0.0005247592926025391), 'calcul

## Find the list of community tools here
https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/community-tools-package/

# Case Study

The remainder of this demo is done in the form of a case study and designed around using a key feature of Bedrock, __Knowledge Bases__.
- *A __Knowledge Base__ is trusted data source enables the AI Agents to provide answers and completing tasks.*

The Federal government requires states to make Wildlife harvest records publicly available. Each State does this on their own Wildlife website making it extremely hard to build a tool that can scrape them reliably. We want to create an Agent that can scrape these sites reliably and upload the data to a knowledge base. We will then create an agent that references that knowledge base in answering questions.

### Goals
1. Given a web page search for PDF files 
2. Upload the pdf files to s3
    - Keep the bucket organized and follow the following naming convention `<animal>/<state>/<year>.pdf`
3. Create a knowledge base
4. Create an agent that references that knowledge base
5. Do not leverage use_aws tool

### Examples
- https://wildlife.utah.gov/hunting/main-hunting-page/big-game/big-game-harvest-data.html
- https://huntillinois.org/harvest-data

#### Use AWS tool
Amazon provides a tool, `use_aws`, that should make this job very easy in theory. In practice this did not work. 

In my first attempt, I tried creating 2 tools to enforce naming consistency and then letting `use_aws` run with it. I knew this solution would not be optimal, but I wanted to see if it would work. The agent could not succeed at this, but I learned quite alot about `use_aws`. My personal recommendation is to avoid this to outside of POC work, it is extremely greedy in how it opperates, deploying resources, failing and not cleaning up after its self. It will continue try and fail and try something new that will fail until it runs out of tokens. This is a great way to run up costs and lower the security of your aws environment. 

This attempt was unable to successfully upload pdf files in every attempt. I would create "pdfs" in s3 but they would be empty files with no contents

## Design Considerations
This agent is designed to only do 1 workflow.
- Find PDF files on a website
- Download the PDF files
- Upload PDFs to S3 
- Create/update the knowledge base

When we think about this workflow, the only AI part of this work is identifying the PDF file urls un a website. Further we can assume that this is going to effectively amount to parsing html. `<a href="<pdf file link>">`
- Government websites especially for hunting can often be behind the times so vanilla html is not uncommon.
- Strands provides a tool `http_request` which we can leverage for these vanilla sites
- More modern Javascript webpages may be more difficult to parse, there are other Strands tools that could improve this in theory, but I couldn't get them configured to run in the time I had. 

The remaining workflow is algorithmic, so instead of leaving it up to the agent we can build tooling for this. Further, we want this to be a singular tool so that there is only 1 possible entry point for the agent to call. If we provided multiple tools we would need to add more instructions (probably numbered instructions) to enforce the order of opperations, but it would be hard to ensure that if a step failed the agent would not do the step after that.


In [ ]:
## you can provide a different bedrock model id here if you want
model = "us.anthropic.claude-sonnet-4-20250514-v1:0"
## provide your name, but keep it short, no more then 6 characters
user ="<put your name here, no spaces>" 
region = "us-east-1"

#### Creating a tool
It's time to create a tool for the AI agent too leverage. The empty function `ingest_files_and_create_knowledge_base` will be the tool that we want the agent to use. 

The function definition and the parameters have been defined

Your objectives are as follows
- Fill out the tool document comment so that the agent can effectively use the tool
- Fill out the contents of the loop in the tools root function so that it calls create_knowledge_base correctly
- Fill out create_knowledge_base function to execute the calls to the necessary knowledge_base management functions

Every area that must be filled out is labelled

In [ ]:
from strands import tool
import requests
import os
from knowledge_base_management import create_knowledge_base_with_s3_vectors, retrieve_knowledge_base, update_knowledge_base_with_s3_vectors
os.environ["BYPASS_TOOL_CONSENT"] = "true"

def download_pdf(url, year):
    response =  requests.get(url)
    local_file_name = year+'.pdf'
    if response.status_code == 200:
        with open(local_file_name, 'wb') as file:
            file.write(response.content)
        print(f"PDF downloaded successfully: {local_file_name}")
        return local_file_name
    else:
        print(f"Failed to download PDF. Status code: {response.status_code}")

def create_knowledge_base(files, topic= f"hunting-workshop-{user}"):
    ## Fill this in Referncing functions in knowledge_base_management.py

@tool
def ingest_files_and_create_knowledge_base(animal:str, state:str, file_details: dict[str, str]) :
    """
    Downloads PDF files from provided URLs, stores them in an S3 bucket, and creates or updates a vector-based Amazon Bedrock knowledge base.

    This tool performs the following steps:
    ###################################### REPLACE THIS LINE AND FILL THIS IN########################################

    Parameters:
    ###################################### REPLACE THIS LINE AND FILL THIS IN########################################

    Returns:
    ###################################### REPLACE THIS LINE AND FILL THIS IN########################################

    Example:
    ###################################### REPLACE THIS LINE AND FILL THIS IN########################################

    """ 
    ## Fill loop here 
    files = []
    for file in file_details.items():
        break
    kb_id = create_knowledge_base(files)
    ## at the end of this function we need to write the kb_id to an environment variable so that it can be referenced later in the jupyter notebook
    os.environ["BEDROCK_KB_ID"] = kb_id
    return kb_id


### The Web Scraping Agent
Fill in the System Prompt and provide the tooling so that the Web Scraping agent will be 

In [ ]:
from strands import Agent
from strands_tools import http_request
import os
os.environ["BYPASS_TOOL_CONSENT"] = "true"


system_prompt = """
"""

hunting_kb_agent = Agent(
    tools=[],
    system_prompt = system_prompt,
    model = model
)

response = hunting_kb_agent("Get the Deer Data from https://wildlife.utah.gov/hunting/main-hunting-page/big-game/big-game-harvest-data.html")


### Using the Knowledge Base

Strands provides us with the tool `retrieve` inorder to tell our agent to use the knowledge base as reference. The agent must be provided with the Knowledge Base's Id. If the default AWS region is not set on your aws account or the region of the knowledge base is not the default then you will need to provide the region in the system prompt.

- In the following cell you have been provded with a base system prompt
- Create an agent that references the knowledge base with this system prompt
- Then modify the system prompt to answer the the user prompt below


In [ ]:
from strands import Agent
import os
os.environ["BYPASS_TOOL_CONSENT"] = "true"
kb_id = os.environ["BEDROCK_KB_ID"]
system_prompt = f"""
You are Hunting guide, your clients will ask you all about hunting. 
[Instructions]
- Search the knowledge base (ID: {kb_id}) in the region {region} and answer questions based on that knowledge base. That knowledge base contains data on Utah and Illinois.
- If you encounter an error accessing the knowledge base print it out to the user
"""
guide_agent = None

response = guide_agent("Historically what regions of Utah have the highest success rates for archery hunters?")


In [ ]:
## Update the system prompt to prevent this question from being answered
response = guide_agent("What color is taylor swifts hair?")

In [ ]:
response = guide_agent("What compare the color of a deer's coat to taylor swifts hair?")

## Lets load some more data
We need to update the Agent tooling to support updating the knowledge base if it already exists.
Go back up to the the cell where you created the tooling four our web scraping agent and make the following updates
- update the tools name say suggest it updates the knowledge base
- update the tools description to say that it updates the knowledge base
- update the tool to check AWS to see if the KB exists (This ability is provided in knowledge_base_management.py)
- update the logic so that it will update the knowledge base if it exists
- Recreate the agent (this needs to be done because you have renamed the tool)

__Disclaimer__: Illinois transitioned from vanilla html to JS while I was working on this, the web scraping agent still works, but I have had it fail once or twice, you may need to rerun the agent if it fails

In [ ]:
response = hunting_kb_agent("Get the Deer Data from https://huntillinois.org/harvest-data")

## Recreating our guide agent

Since we updated our knowledge base in place we don't need to do to recreate the agent, however we want a smarter agent anyway so lets do that now.

Strands has a tools for making agents more robust.
- recreate the guide agent with memory and calculator
- create user prompts to leverage those tools

https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/community-tools-package/

Hint: you shouldn't need to update the system prompt, but you can.

In [ ]:
## recreate the agent here

In [ ]:
## ask a question

In [ ]:
## ask a related question that proves the agent uses memory

In [ ]:
## ask a question that forces our agent to use the calculator

# Making an agent capable of searching the internet

Duck Duck Go provides us with a python library. 
- Create a tool to Leverage the `ddgs.text()` function to preform a keyword search 
- Provide the agent the ability to use the tool 
- Explain to the agent how to leverage tool in the system prompt
- Limit the number of searches the agent can do (this is optional, but annoying if you do not)

In [ ]:
import logging
from strands import Agent, tool
from strands_tools import calculator
from ddgs import DDGS

logging.basicConfig(level=logging.INFO)

@tool
def websearch(
    keywords: str,
    region: str = "us-en",
    max_results: int | None = None,
) -> str:
    """
    """
    try:
        return
    except Exception as e:
        return f"Exception: {e}"

os.environ["BYPASS_TOOL_CONSENT"] = "true"

os.environ["BEDROCK_KB_ID"] = kb_id
system_prompt = f"""
You are Hunting guide, your clients will ask you all about hunting. Do not answer questions unrelated to hunting.
[Instructions]
1. Search the knowledge base (ID: {kb_id}) in the region us-east-1 and answer questions based on that knowledge base.
    - If you encounter an error accessing the knowledge base print it out to the user
###################################### REPLACE THIS LINE AND update this prompt to add search instructions########################################
"""
# Create a Strands agent
better_hunting_guide = Agent(
    name="smart hunting guide",
    system_prompt= system_prompt,
    tools=[retrieve,memory, calculator, websearch],
    callback_handler=None
)

In [ ]:

response = better_hunting_guide("How many deer are hunted in Wisconsin annually?")

# Future add ons

I wanted to add an agent to agent call where the Second Agent tells the first Agent to load Wisconsin's data to the knowledge base. Some of the issues that I was running into are interesting
- not all of the government sites with statistics are .gov 
- Internet searching to find exact source that I want to is a challenge for the agent
- because the first agent can't be counted on to find the proper source how do we solve loading data that may be less reliable to our knowledge base?
